In [4]:
import pandas as pd

## 1. load data

In [5]:
# change this to the path of csv file 
csv_file = '../catalan_data_course/catalan-juvenile-recidivism-subset.csv'
df = pd.read_csv(csv_file, header=0)

In [6]:
df.head()

,id,V1_sex,V4_area_origin,V6_province,V8_age,V9_age_at_program_end,V11_criminal_record,V12_n_criminal_record,V13_n_crime_cat,V15_main_crime_cat,...,V28_days_from_crime_to_program,V29_program_duration,V115_RECID2015_recid,V10_date_of_birth_year,V10_date_of_birth_month,V22_main_crime_date_year,V22_main_crime_date_month,V30_program_start_year,V30_program_start_month,V31_program_end_month
0,691,male,Spain,Lleida,15,17,1,1-2,3+,Against People,...,587.0,132,1,1992,11,2008,8,2010,4,8
1,956,female,Spain,Barcelona,14,15,1,1-2,2,Against People,...,107.0,258,1,1995,7,2009,8,2009,12,8
2,2560,male,Maghreb,Barcelona,15,16,1,3-5,3+,Against People,...,135.0,85,1,1994,1,2009,11,2010,3,6
3,2018,male,Maghreb,Girona,14,15,1,5+,2,Against People,...,176.0,271,1,1994,4,2009,1,2009,6,3
4,1650,male,Spain,Tarragona,17,20,1,5+,3+,Against People,...,261.0,533,1,1990,9,2008,9,2009,5,11


## 2. dataset spliting

In [7]:
import random
random.seed(2023)

# change the percentage here
train_perc,valid_perc,test_perc = 0.6,0.2,0.2

In [8]:
train_set_ids=[]
valid_set_ids=[]
test_set_ids=[]

say if we want to focus on *Area Origin* and *Sex*:

In [9]:
df.groupby('V4_area_origin').count()['id']

V4_area_origin
Europe            244
Latin America     727
Maghreb           610
Other              99
Spain            2972
Name: id, dtype: int64

In [11]:
area_origin_list = ['Spain','Maghreb','Latin America','Europe','Other']
sex_list = ['male','female']


In [12]:
for each_ori in area_origin_list:
    for each_sex in sex_list:
        tmp_list = df.loc[((df['V4_area_origin'] == each_ori) & (df['V1_sex'] == each_sex))]['id'].to_list()
        print('Group{{{},{}}}:{}'.format(each_ori,each_sex,len(tmp_list)))
        
        train_num = int(len(tmp_list)*train_perc)
        valid_num = int(len(tmp_list)*valid_perc)
        test_num = len(tmp_list) - train_num - valid_num
    
    
        train_set_tmp = random.sample(tmp_list,k=train_num)
        valid_set_tmp = random.sample(list(set(tmp_list) - set(train_set_tmp)),k=valid_num)
        test_set_tmp = list(set(tmp_list) - set(train_set_tmp)-set(valid_set_tmp))
    
        train_set_ids.extend(train_set_tmp)
        valid_set_ids.extend(valid_set_tmp)
        test_set_ids.extend(test_set_tmp)

Group{Spain,male}:2338
Group{Spain,female}:634
Group{Maghreb,male}:580
Group{Maghreb,female}:30
Group{Latin America,male}:619
Group{Latin America,female}:108
Group{Europe,male}:193
Group{Europe,female}:51
Group{Other,male}:92
Group{Other,female}:7


In [13]:
df['split'] = df['id'].apply(lambda x: 'train' if x in train_set_ids else ('valid' if x in valid_set_ids else 'test'))

In [14]:
df.head(5)

,id,V1_sex,V4_area_origin,V6_province,V8_age,V9_age_at_program_end,V11_criminal_record,V12_n_criminal_record,V13_n_crime_cat,V15_main_crime_cat,...,V29_program_duration,V115_RECID2015_recid,V10_date_of_birth_year,V10_date_of_birth_month,V22_main_crime_date_year,V22_main_crime_date_month,V30_program_start_year,V30_program_start_month,V31_program_end_month,split
0,691,male,Spain,Lleida,15,17,1,1-2,3+,Against People,...,132,1,1992,11,2008,8,2010,4,8,train
1,956,female,Spain,Barcelona,14,15,1,1-2,2,Against People,...,258,1,1995,7,2009,8,2009,12,8,test
2,2560,male,Maghreb,Barcelona,15,16,1,3-5,3+,Against People,...,85,1,1994,1,2009,11,2010,3,6,valid
3,2018,male,Maghreb,Girona,14,15,1,5+,2,Against People,...,271,1,1994,4,2009,1,2009,6,3,train
4,1650,male,Spain,Tarragona,17,20,1,5+,3+,Against People,...,533,1,1990,9,2008,9,2009,5,11,valid


In [17]:
df.to_csv('../catalan_data_course/catalan-juvenile-recidivism-subset-split.csv')